**Table of contents**<a id='toc0_'></a>    
- [ロボットアーム](#toc1_)    
- [pybulletの起動](#toc2_)    
- [pybulletの初期設定](#toc3_)    
- [ロボットアームの生成](#toc4_)    
- [使用する関数の定義](#toc5_)    
- [カラーのオブジェクトの生成](#toc6_)    
- [カメラの生成](#toc7_)    
- [カメラのパラメータの設定](#toc8_)    
- [画像の取得](#toc9_)    
- [カラーのオブジェクトの位置の推定](#toc10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[ロボットアーム](#toc0_)

本notebookでは6軸のロボットアームを生成し、「固定されたカメラ」から指定したカラーのオブジェクトの位置を推定する方法を紹介します。

（今回は、固定されたカメラから位置を推定するので、アームの制御は行いません。）

（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[pybulletの初期設定](#toc0_)

In [2]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
time_step = 1./240.
pybullet.setTimeStep(time_step) # 1stepあたりに経過する時間の設定

#床の読み込み
plane_id = pybullet.loadURDF("plane.urdf")

# GUIモードの際のカメラの位置などを設定
camera_distance = 3.5
camera_yaw = 180.0 # deg
camera_pitch = -40 # deg
cameraTargetPosition = [0, 0.5, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, cameraTargetPosition)

ven = Mesa
ven = Mesa


# <a id='toc4_'></a>[ロボットアームの生成](#toc0_)

In [3]:
# ロボットの読み込み
arm_start_pos = [0, 0, 0.0]  # 初期位置(x,y,z)を設定
arm_start_orientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
arm_id = pybullet.loadURDF("../urdf/simple6d_arm_with_gripper.urdf", arm_start_pos, arm_start_orientation, useFixedBase=True)



b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc5_'></a>[使用する関数の定義](#toc0_)

In [4]:
import cv2
import numpy as np

def detect_color_obj_pose(target_rgb, rgb_img, depth_img):
    """
    最初に検出された色物体の中心位置、深度、姿勢を取得する関数
    
    Parameters
    ----------
    target_rgb : list
        検出したい色のRGB
    rgb_img : numpy.ndarray
        カメラ画像（RGB）
    depth_img : numpy.ndarray
        カメラ画像（Depth）

    Returns
    -------
    obj_pose : numpy.ndarray
        色物体の位置と姿勢（x, y, z, roll, pitch, yaw）
    """

    # カメラ画像をHSV形式に変換
    hsv_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HSV)

    # RGBをHSVに変換
    target_hsv = cv2.cvtColor(np.uint8([[target_rgb]]), cv2.COLOR_RGB2HSV)[0][0]

    # 検出したい色の範囲を指定
    lower = np.array([target_hsv[0]-10, 50, 50])
    upper = np.array([target_hsv[0]+10, 255, 255])

    # 指定した色のみを抽出
    mask = cv2.inRange(hsv_img, lower, upper)

    # 輪郭を抽出
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 最大面積の輪郭を取得
    max_area = 0
    max_area_contour = None
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            max_area_contour = contour

    # 輪郭が見つからなかった場合
    if max_area_contour is None:
        return None
    
    # 輪郭の中心位置を取得
    M = cv2.moments(max_area_contour)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    # 輪郭の中心位置における深度を取得
    depth = depth_img[cy, cx]

    pos = [cx, cy, depth]
    
    return pos

def Rx(theta):
    """
    x軸周りの回転行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        x軸周りの回転行列
    """
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def Ry(theta):
    """
    y軸周りの回転行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        y軸周りの回転行列
    """
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def Rz(theta):
    """
    z軸周りの回転行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        z軸周りの回転行列
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])


def Hx(theta):
    """
    x軸回りの同次変換行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        x軸回りの同次変換行列
    """
    return np.array([[1, 0, 0, 0],
                     [0, np.cos(theta), -np.sin(theta), 0],
                     [0, np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, 1]])

def Hy(theta):
    """
    y軸回りの同次変換行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        y軸回りの同次変換行列
    """
    return np.array([[np.cos(theta), 0, np.sin(theta), 0],
                     [0, 1, 0, 0],
                     [-np.sin(theta), 0, np.cos(theta), 0],
                     [0, 0, 0, 1]])

def Hz(theta):
    """
    z軸回りの同次変換行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        z軸回りの同次変換行列
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0, 0],
                     [np.sin(theta), np.cos(theta), 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def Hp(x, y, z):
    """
    平行移動の同次変換行列を求める

    Parameters
    ----------
    x : float
        x方向の移動量
    y : float
        y方向の移動量
    z : float
        z方向の移動量

    Returns
    -------
    numpy.ndarray
        平行移動の同次変換行列
    """
    return np.array([[1, 0, 0, x],
                     [0, 1, 0, y],
                     [0, 0, 1, z],
                     [0, 0, 0, 1]])

# <a id='toc6_'></a>[カラーのオブジェクトの生成](#toc0_)

In [5]:
# ここを変えると結果が変わります（simpleCameraの画角に収まるように設定してください）####
color_box_pos = [-2.0, 1.5, 0.05] # 色物体の初期位置(x, y, z)を設定
#################################################################################

color_box_id = pybullet.loadURDF("../urdf/simple_box.urdf", color_box_pos, pybullet.getQuaternionFromEuler([0.0, 0.0, 0.0]), globalScaling=0.1, useFixedBase=True)

# <a id='toc7_'></a>[カメラの生成](#toc0_)

In [6]:
import math

# ここを変えると結果が変わります（colorBoxがsimpleCameraの画角に収まるように設定してください）####
SIMPLE_CAMERA_X = 0.0
SIMPLE_CAMERA_Y = 0.0
SIMPLE_CAMERA_Z = 4.0
SIMPLE_CAMERA_ROLL = 0.0
SIMPLE_CAMERA_PITCH = 0.0
SIMPLE_CAMERA_YAW = 0.0
#################################################################################

SIMPLE_CAMERA_ROLL = math.radians(180.0 + SIMPLE_CAMERA_ROLL)
SIMPLE_CAMERA_PITCH = math.radians(0.0 + SIMPLE_CAMERA_PITCH)
SIMPLE_CAMERA_YAW = math.radians(0.0 + SIMPLE_CAMERA_YAW)
simple_camera_id = pybullet.loadURDF("../urdf/simple_camera.urdf", [SIMPLE_CAMERA_X, SIMPLE_CAMERA_Y, SIMPLE_CAMERA_Z], pybullet.getQuaternionFromEuler([SIMPLE_CAMERA_ROLL, SIMPLE_CAMERA_PITCH, SIMPLE_CAMERA_YAW]), useFixedBase=True)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: base_link
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc8_'></a>[カメラのパラメータの設定](#toc0_)

In [7]:
# カメラ設定
fov = 60
image_width = 224
image_height = 224
aspect = image_width / image_height
near = 0.05
far = 5
projection_matrix = pybullet.computeProjectionMatrixFOV(fov, aspect, near, far)

# 焦点距離を求める
fov_rad = np.deg2rad(fov)
f = (image_height / 2) / np.tan(fov_rad / 2)

# カメラの内部パラメータ
camera_matrix = np.array([[f, 0, image_width//2],
                         [0, f, image_height//2],
                         [0, 0, 1]], dtype=np.float32)

# 歪み係数（ここでは、歪みがないと仮定）
dist_coeffs = np.array([0.0, 0.0, 0.0, 0.0, 0.0], dtype=np.float32)

# <a id='toc9_'></a>[画像の取得](#toc0_)


In [8]:
SIMPLE_CAMERA_LINK_IDX = 0
SIMPLE_CAMERA_TARGET_LINK_IDX = 1

# カメラの位置を取得
camera_link_pose = pybullet.getLinkState(simple_camera_id, SIMPLE_CAMERA_LINK_IDX)[0] # 手先のカメラリンクの位置
camera_target_link_pose = pybullet.getLinkState(simple_camera_id, SIMPLE_CAMERA_TARGET_LINK_IDX)[0] # カメラリンクの少しだけ前に設定した仮想的なリンクの位置
camera_link_orientation = pybullet.getEulerFromQuaternion(pybullet.getLinkState(simple_camera_id, SIMPLE_CAMERA_LINK_IDX)[1]) # 手先のカメラリンクの姿勢

# カメラの姿勢に合わせてカメラの上方向のベクトルを回転
camera_up_vector = np.array([0, -1, 0]) # デフォルトのカメラの上方向のベクトル
R = Rz(SIMPLE_CAMERA_YAW)@Ry(SIMPLE_CAMERA_PITCH)@Rx(SIMPLE_CAMERA_ROLL)
rotate_camera_up_vector = R@camera_up_vector

# カメラのビュー行列を計算
view_matrix = pybullet.computeViewMatrix(cameraEyePosition=[camera_link_pose[0], camera_link_pose[1], camera_link_pose[2]], cameraTargetPosition=[camera_target_link_pose[0], camera_target_link_pose[1], camera_target_link_pose[2]],cameraUpVector=[rotate_camera_up_vector[0], rotate_camera_up_vector[1], rotate_camera_up_vector[2]])

# カメラ画像を取得
_, _, rgb_img, depth_img, _ = pybullet.getCameraImage(
    width=image_width,
    height=image_height,
    viewMatrix=view_matrix,
    projectionMatrix=projection_matrix,
    renderer=pybullet.ER_BULLET_HARDWARE_OPENGL
)

# 指定した色の物体の位置を取得
detect_color_rgb = [0, 0, 255] # 検出したい色を指定（ここでは赤色）
pos = detect_color_obj_pose(detect_color_rgb, rgb_img, depth_img)
pixel_x = pos[0] # x座標（画像上の位置）
pixel_y = pos[1] # y座標（画像上の位置）
normal_z = pos[2] # z座標（0～1に正規化された深度）
z = far * near / (far - (far - near) * normal_z) # 0～1に正規化された深度を距離[m]に変換
x = (pixel_x - image_width//2) * z / f # pixelXを距離[m]に変換
y = (pixel_y - image_height//2) * z / f # pixelYを距離[m]に変換

# <a id='toc10_'></a>[カラーのオブジェクトの位置の推定](#toc0_)

In [9]:
# ワールド座標系におけるカラーの物体の位置を計算
CAMERA_POS = np.array([SIMPLE_CAMERA_X, 
                      SIMPLE_CAMERA_Y, 
                      SIMPLE_CAMERA_Z])
color_obj_pos_camera = np.array([x, y, z]) # カメラ座標系における物体の位置
color_obj_pos_world = CAMERA_POS + R@color_obj_pos_camera # ワールド座標系における物体の位置

# ボックスの位置を取得
box_pos, box_orn = pybullet.getBasePositionAndOrientation(color_box_id)
print("color box obj pose: ", box_pos[0], box_pos[1], box_pos[2])
print("estimate color obj pose", color_obj_pos_world[0], color_obj_pos_world[1], color_obj_pos_world[2])

# Pybuletの画面上に、物体の位置を描画
pybullet.addUserDebugLine(CAMERA_POS[:3], color_obj_pos_world[:3], lineColorRGB=[1, 0, 0], lineWidth=5)

# ボックスの位置を描画
pybullet.addUserDebugText(f"true box pose ({box_pos[0]:.3f}, {box_pos[1]:.3f}, {box_pos[2]:.3f})", [4.0, 0.5, 0], textColorRGB=[1, 0, 0], textSize=1.3)
pybullet.addUserDebugText(f"eye to hand, estimate box pose ({color_obj_pos_world[0]:.3f}, {color_obj_pos_world[1]:.3f}, {color_obj_pos_world[2]:.3f})", [4.0, 1.0, 0.0], textColorRGB=[1, 0, 0], textSize=1.3)

color box obj pose:  -2.0 1.5 0.05
estimate color obj pose -1.9903058737195025 1.5078074800905317 0.10001191576652735


2